In [ ]:
import warnings
warnings.filterwarnings('ignore')

https://machinelearningmastery.com/best-practices-document-classification-deep-learning/

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn import model_selection
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from gensim.models import Phrases, Word2Vec

In [ ]:
#from nltk import word_tokenize,pos_tag

In [ ]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
import matplotlib.pyplot as plt
from IPython.display import SVG
from sklearn.manifold import TSNE

In [ ]:
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, Flatten, LSTM,Convolution1D,Dropout

from keras.layers.convolutional import MaxPooling1D, MaxPooling2D

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence,text as Text
from keras.utils import vis_utils #import plot_model, model_to_dot

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

google_pretrained_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def cleanTweets(texts):
    cleanedTexts = []
    for text in texts:
        cleanedTexts += [unicode(str(text),errors='ignore').encode("latin-1","ignore").replace('\r','')]
    return cleanedTexts

https://www.kaggle.com/marijakekic/cnn-in-keras-with-pretrained-word2vec-weights

In [ ]:
df = pd.read_csv('agressive_tweet_classed.txt',sep='==>',encoding='utf-8')

tweets = cleanTweets(df['tweet'].tolist())
labels = df['class'].tolist()

In [ ]:
def plot_model(model):
    return SVG(vis_utils.model_to_dot(model).create(prog='dot', format='svg'))

def generateEmbeddingMatrix(word_vectors, embedding, word_index, vocabulary_size=vocab_size):
    NUM_WORDS = vocabulary_size
    embedding_dim = len(embedding[embedding.keys()[0]])
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    for word, i in word_index.items():
        if i>=NUM_WORDS:
            continue
        try:
            embedding_vector = word_vectors[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
    return embedding_matrix

### <font color="bieg">Preparing Google PreTrained Embeddings</font>

In [ ]:
EMBEDDING_DIM=300
NUM_WORDS = vocab_size
google_embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        google_embedding_vector = google_pretrained_vectors[word]
        google_embedding_matrix[i] = google_embedding_vector
    except KeyError:
        google_embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(google_pretrained_vectors)

### <font color="orange"> Twitter Glove Embedding </font>

In [ ]:
embeddings_index = {}
word_index = tokenizer.word_index

for line in open('glove/glove.twitter.27B.50d.txt'):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

glove_twitter_embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_twitter_embedding_matrix[i] = embedding_vector

In [ ]:
embeddings_index = {}
word_index = tokenizer.word_index

for line in open('glove/glove.twitter.27B.200d.txt'):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

glove_twitterBig_embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_twitterBig_embedding_matrix[i] = embedding_vector

In [ ]:
def plotTestTrainCurves(history):
    print "History Summary for accuracy"
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    print "History Summary for loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def train_test_split(X=tweets,y=labels):
    X = np.array(X)
    y = np.array(y)
    ###################################
    X,y = shuffle(X,y,random_state=10)
    X,y = shuffle(X,y,random_state=0)
    ###################################
    tk = Text.Tokenizer(lower=True)
    tk.fit_on_texts(X)

    x = tk.texts_to_sequences(X)

    ###################################
    max_len = 174
    embedding_vecor_length = 32
    print "max_len ", max_len
    print('Pad sequences (samples x time)')

    x = sequence.pad_sequences(x, maxlen=max_len)

    return model_selection.train_test_split(x,y, test_size=0.2)

def scores(y_test,y_pred):
    print 
    print("Accuracy: %.2f%%" % (accuracy_score(y_test,y_pred)*100))
    print("Precision: %.2f%%" % (precision_score(y_test,y_pred)*100))
    print("Recall: %.2f%%" % (recall_score(y_test,y_pred)*100))
    print("F1-score: %.2f%%" % (f1_score(y_test,y_pred)*100))

In [ ]:
def find_max_length(texts=tweets):
    maxLength = -1
    for text in texts:
        if len(text)>maxLength:
            maxLength = len(text)
    return maxLength

https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model

In [ ]:
def Words(sentences):
    words = []
    for sentence in sentences:
        tokens =  word_tokenize(sentence)
        words +=[tokens]
    return words

In [ ]:
tweetCorpus = cleanTweets(open('profane_tweets.txt').readlines())

In [ ]:
word2vec_transformer = Words(tweetCorpus)
phrase2vec_transformer = Phrases(tweetCorpus)

In [ ]:
cbow_word_model = Word2Vec(word2vec_transformer, size=100)
CBOW_word_W2V = dict(zip(cbow_word_model.wv.index2word, cbow_word_model.wv.syn0))

skipgram_word_model = Word2Vec(word2vec_transformer, sg=1, size=100)
skipgram_word_W2V = dict(zip(skipgram_word_model.wv.index2word, skipgram_word_model.wv.syn0))

In [ ]:
tokenizer = Text.Tokenizer()
tokenizer.fit_on_texts(tweets)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
vocab_size

In [ ]:
max_profane_length = find_max_length()
max_profane_length

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()

In [ ]:
labels.count(0), labels.count(1)

### 1-D CNN model without Maxpooling
https://medium.com/@thoszymkowiak/how-to-implement-sentiment-analysis-using-word-embedding-and-convolutional-neural-networks-on-keras-163197aef623

In [ ]:
embedding_vector_length = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_profane_length))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plotTestTrainCurves(history)

In [ ]:
embedding_vector_length = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_profane_length))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.1, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### word2vec

In [ ]:
embedding_matrix = generateEmbeddingMatrix(cbow_word_model, CBOW_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(CBOW_word_W2V[CBOW_word_W2V.keys()[0]])

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
earlystopping_nopatience = [EarlyStopping(monitor='val_loss')]
earlystopping_patience = [EarlyStopping(monitor='val_loss',patience=1)]

In [ ]:
embedding_matrix = generateEmbeddingMatrix(cbow_word_model, CBOW_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(CBOW_word_W2V[CBOW_word_W2V.keys()[0]])
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### Training on Google Pre-trained model

#### trainable = True

In [ ]:
embedding_matrix = google_embedding_matrix
embedding_dim = 300

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, callbacks = earlystopping, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

#### trainable = False

In [ ]:
embedding_matrix = google_embedding_matrix
embedding_dim = 300

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=False))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, callbacks = earlystopping_patience, shuffle=True)
# Final evaluation of the model
predicted = model.predict_classes(X_test)
scores(y_test,predicted)
plotTestTrainCurves(history)

In [ ]:
embedding_matrix = google_embedding_matrix
embedding_dim = 300

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(512, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, callbacks = earlystopping, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### Skipgrams with negative sampling (negative = 20 ~ default = 5)

In [ ]:
skipgramNeg_word_model = Word2Vec(word2vec_transformer, sg=1, size=100,negative=20)
skipgramNeg_word_W2V = dict(zip(skipgramNeg_word_model.wv.index2word, skipgramNeg_word_model.wv.syn0))

In [ ]:
embedding_matrix = generateEmbeddingMatrix(skipgramNeg_word_model, skipgramNeg_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(skipgram_word_W2V[skipgram_word_W2V.keys()[0]])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
embedding_matrix = generateEmbeddingMatrix(skipgramNeg_word_model, skipgramNeg_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(skipgram_word_W2V[skipgram_word_W2V.keys()[0]])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
embedding_matrix = generateEmbeddingMatrix(skipgramNeg_word_model, skipgramNeg_word_W2V,tokenizer.word_index, vocabulary_size=vocab_size)
embedding_dim = len(skipgram_word_W2V[skipgram_word_W2V.keys()[0]])

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
skipgram_phrase_model = Word2Vec(phrase2vec_transformer[tweetCorpus], sg=1, size=100, negative=20)
skipgram_phrase_W2V = dict(zip(skipgram_phrase_model.wv.index2word, skipgram_phrase_model.wv.syn0))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 5, border_mode='same'))
model.add(Convolution1D(32, 5, border_mode='same'))
model.add(Convolution1D(16, 5, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 5, border_mode='same'))
model.add(Convolution1D(32, 5, border_mode='same'))
model.add(Convolution1D(16, 5, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim,  weights=[embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 5, border_mode='same'))
model.add(Convolution1D(32, 5, border_mode='same'))
model.add(Convolution1D(16, 5, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.summary()
X_train, X_test, y_train, y_test = train_test_split()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1, validation_split=0.1, callbacks=earlystopping, shuffle=True)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### <font color="garden">Single Layer CNN</font>

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50,  weights=[glove_twitter_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50,  weights=[glove_twitter_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(100,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50,  weights=[glove_twitter_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(2048,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### <font color="inkblue">Twitter Glove with CNN</font>

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50,  weights=[glove_twitter_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 200,  weights=[glove_twitterBig_embedding_matrix], input_length=max_profane_length,trainable=True))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.1, shuffle=True,callbacks=earlystopping)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

### Things to tinker with 
1. How to decide on how much dense it should be 
2. Reduce dropout before dense layer 

### 1-D CNN model with Maxpooling

In [ ]:
embedding_vector_length = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_profane_length))
model.add(Convolution1D(256, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Convolution1D(128, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Convolution1D(32, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, callbacks = earlystopping, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

In [ ]:
embedding_vector_length = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_profane_length))
model.add(Convolution1D(256, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Convolution1D(128, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Convolution1D(64, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Convolution1D(16, 3, activation='relu', border_mode='same'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1, callbacks = earlystopping, validation_split=0.1, shuffle=True)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
plotTestTrainCurves(history)

http://phontron.com/class/nn4nlp2017/assets/slides/nn4nlp-05-cnn.pdf

### 2D Convolutional Networks 

In [ ]:
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5


inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

model.add(Convolution2D(64, 3, border_mode='same'))
model.add(Convolution2D(32, 3, border_mode='same'))
model.add(Convolution2D(16, 3, border_mode='same'))

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)

In [ ]:
adam = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, y_train, batch_size=1000, epochs=10, verbose=1, validation_data=(X_val, y_val),
         callbacks=callbacks)  # starts training